## Training your own model with Tensorflow!

This exercise will done using Google Colab, Python, and Tensorflow.



Please click on the play button below to make sure you have the necessary libraries.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences


In this lesson, we'll be classifying movie reviews as negative or positive based off IMDB's dataset! I don't know how many of you have heard of IMDB, but it's a popular movie database website where users can rate movies.

Press the button below to download the dataset onto this Colab instance! It won't download to your actual machine.
It may take 5s-1m to download.

In [ ]:

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
print(f"Training Sample Count: {len(x_train)}")
print(f"Test Sample Count: {len(x_test)}")

Training Sample Count: 25000
Test Sample Count: 25000


This dataset has been split into train & test subsets.
The train subset is used to build up the model. The test subset is not used to make the model directly, but rather to test if it's accurate.

You should always have a train & test dataset to make sure your data is accurate. In this case, 50% of the data is used for testing and 50% is used for training.

To make the dataset in a format that Tensorflow can use, we need to play around with the text in the dataset. This command makes sure that no review is more than 256 characters.

---



In [ ]:
x_train = pad_sequences(x_train, maxlen=256)
x_test = pad_sequences(x_test, maxlen=256)


Shape of training data: (25000, 256)


Now let's actually create the model!

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=256),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

This might be a bit difficult to understand.
When words are input into this model, they aren't actually recognized by the model as words, but numbers. This allows it to more easily group words with similar meaning together e.g. "mad" and "angry" together to make it easier for the model to recognize.

Flatten and Dense are outside of scope of this lesson to explain, but feel feel free to look them up if interested!

Now we'll train the model using the training data we downloaded earlier! This may take a while. If you want it to go faster, you can decrease the number of epochs (min times it runs through every value in the dataset) but that may affect accuracy!

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.7034 - loss: 0.5185 - val_accuracy: 0.8594 - val_loss: 0.3303
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.9630 - loss: 0.1138 - val_accuracy: 0.8498 - val_loss: 0.3852
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.9966 - loss: 0.0178 - val_accuracy: 0.8523 - val_loss: 0.4725
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.9997 - loss: 0.0025 - val_accuracy: 0.8548 - val_loss: 0.5185
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 1.0000 - loss: 4.3979e-04 - val_accuracy: 0.8570 - val_loss: 0.5434


Now that our model is trained, it's time to see if it can correctly classify our test data!




In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Accuracy: {round(test_accuracy*100, 2)}%')


782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8558 - loss: 0.5449
Accuracy: 85.7%


The accuracy will change each time you complete the sequence, but for me it was 85.7% accurate. Not bad!

Don't focus on the loss value for now; it measures how well the model's predictions fit their regular values. Unlike accuracy, it isn't a percentage, so don't think of it that way and you can ignore it for the purposes of this exercise.

# Model In Action

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(encoded_review):
    return " ".join([reverse_word_index.get(i - 3, "?") for i in encoded_review])

def visualize_prediction(review, actual_label, model):
    padded_review = pad_sequences([review], maxlen=256)
    prediction = model.predict(padded_review)[0][0]
    actual_label = "Positive" if actual_label == 1 else "Negative"
    predicted_label = "Positive" if prediction > 0.5 else "Negative"

    print(f"Review: {decode_review(review)[:257]}{'...' if len(decode_review(review)) > 256 else ''}")
    print(f"Actual: {actual_label}")
    print(f"Predicted: {predicted_label}")
    print(f"Confidence: {round(prediction, 2)*100}% chance it's Positive")

for i in range(5):
    visualize_prediction(x_test[i], y_test[i], model)
    print('\n')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Review: ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out...
Actual: Negative
Predicted: Negative
Confidence: 0.0% chance it's Positive


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Review: ? this film requires a lot of patience because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances ? the sandy dennis character apartment but the film builds to a disturbing clima...
Actual: Positive
Predicted: Positive
Confidence: 100.0% chance it's Positive


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Review: ? many animation buffs consider ? ? the great forgotten genius of one special branch of the art puppet animation which he invented almost single ? and as it happened almost accidentall

As mentioned before, the text is actually processed as numbers rather than words directly. The decode function converts the text back into words so humans can understand it.
The confidence represents the percentage the model is sure the prediction is positive.

Type your own review here and see how the model categorizes it!

In [ ]:
def predict_custom_text(model, text):
    tokens = [word_index.get(word, 2) + 3 for word in text.lower().split()]
    padded_tokens = pad_sequences([tokens], maxlen=256)

    prediction = model.predict(padded_tokens)[0][0]
    predicted_label = "Positive" if prediction > 0.5 else "Negative"
    print(f"Text: {text}")
    print(f"Predicted Label: {predicted_label}")
    print(f"Confidence: {round(prediction)*2}% it's Positive")

user_text = input("Type your own review: ")
predict_custom_text(model, user_text)


Type your own review: Man this movie sucks! I hate Star Wars and anything having to do with it
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Text: Man this movie sucks! I hate Star Wars and anything having to do with it
Predicted Label: Positive
Confidence: 2% it's Positive
